In [1]:
import pandas as pd
from sqlalchemy import create_engine

## Data Cleanup & Analysis

Once you have identified your datasets, perform ETL on the data. Make sure to plan and document the following:
* The sources of data that you will extract from.
    CDC data csv's on autism rates by state and mmr vaccinations by state
* The type of transformation needed for this data (cleaning, joining, filtering, aggregating, etc).
    converting xls into csv
    removing extra header rows
    
* The type of final production database to load the data into (relational or non-relational).
    postgres, relational database
* The final tables or collections that will be used in the production database.
You will be required to submit a final technical report with the above information and steps required to reproduce your ETL process.

## Project Report

At the end of the week, your team will submit a Final Report that describes the following:
* **E**xtract: your original data sources and how the data was formatted (CSV, JSON, pgAdmin 4, etc).
* **T**ransform: what data cleaning or transformation was required.
* **L**oad: the final database, tables/collections, and why this was chosen.
Please upload the report to Github and submit a link to Bootcampspot.


In [11]:
#read in csv after doing some initial clean up of source data
mmr_file = "mmr_data_2.csv"
mmr_data = pd.read_csv(mmr_file)
mmr_data.head()

,Names,2000,LL,UL,CI,SAMPLE SIZE,TARGET,DIFFERENCE TO PREVIOUS YEAR,2001,LL.1,...,UL.16,CI.16,SAMPLE SIZE.16,TARGET.16,2017,LL.17,UL.17,CI.17,SAMPLE SIZE.17,TARGET.17
0,Alabama,86.8,81.8,90.5,(±4.3),580.0,90.0,NaN,94.3,91.4,...,94.8,(±4.8),249.0,90.0,92.3,87.5,95.4,(±3.8),295.0,90.0
1,AL-Jefferson County,87.4,81.9,91.4,(±4.7),294.0,90.0,NaN,93.2,88.3,...,NaN,NaN,NaN,90.0,NaN,NaN,NaN,NaN,NaN,90.0
2,AL-Rest of State,86.6,80.8,90.9,(±5),286.0,90.0,NaN,94.5,91.1,...,NaN,NaN,NaN,90.0,NaN,NaN,NaN,NaN,NaN,90.0
3,Alaska,87.9,83.3,91.4,(±4),295.0,90.0,NaN,87.8,83.1,...,89.8,(±4.8),288.0,90.0,89.3,84.6,92.7,(±4),251.0,90.0
4,Arizona,88.1,84.7,90.7,(±3),595.0,90.0,NaN,88.8,85.5,...,92.3,(±5.7),230.0,90.0,91.7,87.0,94.8,(±3.9),265.0,90.0


In [27]:
# filter down to columns we want to use
mmr_data_filtered = mmr_data[['Names','2000','SAMPLE SIZE','2001','SAMPLE SIZE.1','2002','SAMPLE SIZE.2','2003','SAMPLE SIZE.3','2004','SAMPLE SIZE.4','2005','SAMPLE SIZE.5','2006','SAMPLE SIZE.6','2007','SAMPLE SIZE.7','2008','SAMPLE SIZE.8','2009','SAMPLE SIZE.9','2010','SAMPLE SIZE.10','2011','SAMPLE SIZE.11','2012','SAMPLE SIZE.12','2013','SAMPLE SIZE.13','2014','SAMPLE SIZE.14','2015','SAMPLE SIZE.15','2016','SAMPLE SIZE.16','2017','SAMPLE SIZE.17']].copy()
mmr_data_filtered.head()

,Names,2000,SAMPLE SIZE,2001,SAMPLE SIZE.1,2002,SAMPLE SIZE.2,2003,SAMPLE SIZE.3,2004,...,2013,SAMPLE SIZE.13,2014,SAMPLE SIZE.14,2015,SAMPLE SIZE.15,2016,SAMPLE SIZE.16,2017,SAMPLE SIZE.17
0,Alabama,86.8,580.0,94.3,592.0,91.6,525.0,91.2,532.0,91.8,...,89.7,166.0,92.0,237.0,95.2,272.0,91.1,249.0,92.3,295.0
1,AL-Jefferson County,87.4,294.0,93.2,277.0,91.1,258.0,94.1,252.0,89.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AL-Rest of State,86.6,286.0,94.5,315.0,91.7,267.0,90.7,280.0,92.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Alaska,87.9,295.0,87.8,292.0,88.7,262.0,90.7,278.0,89.7,...,90.5,299.0,90.2,276.0,89.7,295.0,85.6,288.0,89.3,251.0
4,Arizona,88.1,595.0,88.8,592.0,88.9,558.0,91.5,612.0,92.9,...,91.4,254.0,84.1,292.0,90.6,266.0,87.7,230.0,91.7,265.0


In [28]:
mmr_data_filtered_again = mmr_data_filtered[~mmr_data_filtered['Names'].str.contains('-')]
mmr_data_filtered_again.head()

,Names,2000,SAMPLE SIZE,2001,SAMPLE SIZE.1,2002,SAMPLE SIZE.2,2003,SAMPLE SIZE.3,2004,...,2013,SAMPLE SIZE.13,2014,SAMPLE SIZE.14,2015,SAMPLE SIZE.15,2016,SAMPLE SIZE.16,2017,SAMPLE SIZE.17
0,Alabama,86.8,580.0,94.3,592.0,91.6,525.0,91.2,532.0,91.8,...,89.7,166.0,92.0,237.0,95.2,272.0,91.1,249.0,92.3,295.0
3,Alaska,87.9,295.0,87.8,292.0,88.7,262.0,90.7,278.0,89.7,...,90.5,299.0,90.2,276.0,89.7,295.0,85.6,288.0,89.3,251.0
4,Arizona,88.1,595.0,88.8,592.0,88.9,558.0,91.5,612.0,92.9,...,91.4,254.0,84.1,292.0,90.6,266.0,87.7,230.0,91.7,265.0
7,Arkansas,88.0,298.0,89.1,392.0,92.8,286.0,90.6,253.0,94.6,...,88.3,188.0,89.1,245.0,90.2,229.0,92.1,229.0,92.6,258.0
8,California,89.4,1241.0,90.8,1266.0,90.4,1101.0,92.8,1074.0,93.1,...,90.7,283.0,90.5,300.0,92.8,345.0,89.3,328.0,92.9,324.0


In [29]:
#reset index to the state names
mmr_data_filtered_again.set_index(keys='Names', inplace=True)

In [30]:
mmr_data_filtered_again.head()

,2000,SAMPLE SIZE,2001,SAMPLE SIZE.1,2002,SAMPLE SIZE.2,2003,SAMPLE SIZE.3,2004,SAMPLE SIZE.4,...,2013,SAMPLE SIZE.13,2014,SAMPLE SIZE.14,2015,SAMPLE SIZE.15,2016,SAMPLE SIZE.16,2017,SAMPLE SIZE.17
Names,,,,,,,,,,,,,,,,,,,,,
Alabama,86.8,580.0,94.3,592.0,91.6,525.0,91.2,532.0,91.8,515.0,...,89.7,166.0,92.0,237.0,95.2,272.0,91.1,249.0,92.3,295.0
Alaska,87.9,295.0,87.8,292.0,88.7,262.0,90.7,278.0,89.7,299.0,...,90.5,299.0,90.2,276.0,89.7,295.0,85.6,288.0,89.3,251.0
Arizona,88.1,595.0,88.8,592.0,88.9,558.0,91.5,612.0,92.9,603.0,...,91.4,254.0,84.1,292.0,90.6,266.0,87.7,230.0,91.7,265.0
Arkansas,88.0,298.0,89.1,392.0,92.8,286.0,90.6,253.0,94.6,270.0,...,88.3,188.0,89.1,245.0,90.2,229.0,92.1,229.0,92.6,258.0
California,89.4,1241.0,90.8,1266.0,90.4,1101.0,92.8,1074.0,93.1,1144.0,...,90.7,283.0,90.5,300.0,92.8,345.0,89.3,328.0,92.9,324.0


In [5]:
# read in csv
autism_file = "autism_data.csv"
autism_data = pd.read_csv(autism_file)
autism_data.head()

,year,abbr,male.prev,male.ci_l,male.ci_u,female.prev,female.ci_l,female.ci_u,nhw.prev,nhw.ci_l,nhw.ci_u,nhb.prev,nhb.ci_l,nhb.ci_u,his.prev,his.ci_l,his.ci_u,api.prev,api.ci_l,api.ci_u
0,2000,AZ,9.7,8.5,11.1,3.2,2.5,4.0,8.6,7.5,9.8,7.3,4.4,12.2,No data,No data,No data,No data,No data,No data
1,2000,GA,11.0,9.7,12.4,2.0,1.5,2.7,7.9,6.7,9.3,5.3,4.4,6.4,No data,No data,No data,No data,No data,No data
2,2000,MD,8.6,7.1,10.6,2.2,1.5,2.7,4.9,3.8,6.4,6.1,4.7,8,No data,No data,No data,No data,No data,No data
3,2000,NJ,14.8,13.0,16.8,4.3,3.3,5.5,11.3,9.5,13.3,10.6,8.5,13.1,No data,No data,No data,No data,No data,No data
4,2000,SC,9.3,7.8,11.2,3.3,2.4,4.5,6.5,5.2,8.2,5.8,4.5,7.3,No data,No data,No data,No data,No data,No data


In [16]:
autism_data_filtered = autism_data[['year', 'abbr', 'male.prev', 'female.prev']].copy()
autism_data_filtered.head()

,year,abbr,male.prev,female.prev
0,2000,AZ,9.7,3.2
1,2000,GA,11.0,2.0
2,2000,MD,8.6,2.2
3,2000,NJ,14.8,4.3
4,2000,SC,9.3,3.3
